# tei

> TEI Client

In [ ]:
#| default_exp tei

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
from bs4 import BeautifulSoup
from xml.dom.minidom import parseString
import os

In [ ]:
#| export
class TEIClient:
    def convert_manifest3_annotations_to_zones(self, input_iiif_manifest_file_path: str, output_tei_xml_file_path: str) -> None:
        """
        ManifestファイルをTEIのXML形式に変換するメソッド

        Args:
            input_iiif_manifest_file_path (str): 入力となるManifestファイルのパス
            output_tei_xml_file_path (str): 出力となるTEIのXMLファイルのパス

        Returns:
            None
        """

        manifest = self._load_json(input_iiif_manifest_file_path)

        annotations = manifest['items'][0]["annotations"][0]["items"]

        # XMLテンプレートの作成
        xml_string = self._create_initial_xml(manifest)

        # BeautifulSoupでXML操作
        soup = BeautifulSoup(xml_string, 'xml')
        self._append_zones_to_xml(soup, annotations)

        # XMLを整形してファイルに書き出し
        self._write_formatted_xml(soup, output_tei_xml_file_path)

    def _load_json(self, path: str) -> dict:
        """
        JSONファイルを読み込むメソッド

        Args:
            path (str): JSONファイルのパス

        Returns:
            dict: JSONファイルの内容
        """

        with open(path, 'r') as f:
            return json.load(f)
        
    def _create_initial_xml(self, manifest: dict) -> str:
        """
        初期のXMLテンプレートを作成するメソッド

        Args:
            manifest (dict): Manifestファイルの内容
        
        Returns:
            str: 初期のXMLテンプレート
        """


        graphic_url = manifest['items'][0]["items"][0]["items"][0]['body']['id']
        return f'''<?xml version="1.0" encoding="UTF-8"?>
            <TEI xmlns="http://www.tei-c.org/ns/1.0">
                <teiHeader>
                    <fileDesc>
                        <titleStmt>
                            <title>Example</title>
                        </titleStmt>
                        <publicationStmt>
                            <p>Example</p>
                        </publicationStmt>
                        <sourceDesc>
                            <p>Example</p>
                        </sourceDesc>
                    </fileDesc>
                    <profileDesc>
                        <creation>
                            <listChange>
                                <change></change>
                            </listChange>
                        </creation>
                    </profileDesc>
                </teiHeader>
                <facsimile>
                    <surface>
                        <graphic url="{graphic_url}"></graphic>
                    </surface>
                </facsimile>    
            </TEI>'''
    
    def _append_zones_to_xml(self, soup: BeautifulSoup, annotations: list) -> None:
        """
        BeautifulSoupを使用してzoneタグを追加するメソッド

        Args:
            soup (BeautifulSoup): BeautifulSoupオブジェクト
            annotations (list): Manifestファイルのannotationsのリスト
        
        Returns:
            None
        """

        surface = soup.find('surface')

        for i, annotation in enumerate(annotations):
            zone = soup.new_tag('zone')
            zone['xml:id'] = f"a_{str(i).zfill(4)}"
            xywh = annotation['target'].split("#xywh=")[1]
            coords = xywh.split(',')

            # 座標情報の設定
            zone['ulx'], zone['uly'] = coords[0], coords[1]
            zone['lrx'] = str(int(coords[0]) + int(coords[2]))
            zone['lry'] = str(int(coords[1]) + int(coords[3]))

            # テキスト内容の設定
            text = annotation['body']['value']
            text = text.split("]")[1].strip() if "]" in text else text
            seg = soup.new_tag('seg')
            seg.string = text
            zone.append(seg)

            surface.append(zone)

    # 整形したXMLをファイルに書き出す
    def _write_formatted_xml(self, soup: BeautifulSoup, opath: str) -> None:
        """
        整形したXMLをファイルに書き出す



        
        """

        formatted_str = self._format_xml_minidom(str(soup))
        os.makedirs(os.path.dirname(opath), exist_ok=True)
        with open(opath, 'w') as f:
            f.write(formatted_str)

    # minidomを使用してXMLを整形する
    def _format_xml_minidom(self, xml_string: str) -> str:
        """
        minidomを使用してXMLを整形する

        """

        dom = parseString(xml_string)
        pretty_xml_as_string = dom.toprettyxml()

        # 空行を除去
        return '\n'.join(line for line in pretty_xml_as_string.split('\n') if line.strip())

In [ ]:
show_doc(TEIClient.convert_manifest3_annotations_to_zones)

---

[source](https://github.com/nakamura196/iiif_tei_py/blob/main/iiif_tei_py/tei.py#L16){target="_blank" style="float:right; font-size:smaller"}

### TEIClient.convert_manifest3_annotations_to_zones

>      TEIClient.convert_manifest3_annotations_to_zones
>                                                        (input_iiif_manifest_fi
>                                                        le_path:str, output_tei
>                                                        _xml_file_path:str)

ManifestファイルをTEIのXML形式に変換するメソッド

Args:
    input_iiif_manifest_file_path (str): 入力となるManifestファイルのパス
    output_tei_xml_file_path (str): 出力となるTEIのXMLファイルのパス

Returns:
    None

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()